In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import Loess
import importlib
importlib.reload(Loess)

In [ ]:
data = pd.DataFrame(columns=['x', 'y'])
puntos = 50
x = np.linspace(-5, 5, puntos)
y = np.linspace(-5, 5, puntos)
xx, yy = np.meshgrid(x, y)
data["x"] = xx.flatten()
data["y"] = yy.flatten()

In [ ]:
def plano(x, y):
    return -3 * x + y

def silla(x, y):
    return (x**2) - (y**2)

def cubica(x, y):
    return -(x**3) + y

def modulo(x, y):
    return abs(x) + abs(y)

In [ ]:
superficies = {
    "plano": plano,
    "silla": silla,
    "cubica": cubica,
    "modulo": modulo
}

In [ ]:
def agregar_superficie(data, xx, yy, nombre, f):
    zz = f(xx, yy)
    data[nombre] = zz.flatten()
    noise = np.random.normal(0, zz.std() * .5, zz.shape)
    zz = zz + noise
    data[f"{nombre}_ruido"] = zz.flatten()

for nombre, funcion in superficies.items():
    agregar_superficie(data, xx, yy, nombre, funcion)
data.head()

In [ ]:
def agregar_estimacion(loess, data, nombre, f):
    X = data[["x", "y"]]
    estimaciones_1 = []
    estimaciones_2 = []
    for i in range(len(X)):
        Y = data[[f"{nombre}_ruido"]]
        loess.reset_yy(Y)
        x = np.array(X.iloc[i])

        loess.reset_degree(1)
        estimaciones_1.append(loess.estimate(x, f))
        loess.reset_degree(2)
        estimaciones_2.append(loess.estimate(x, f))
    data[f"estimacion_{nombre}_{f}_1"] = estimaciones_1
    data[f"estimacion_{nombre}_{f}_2"] = estimaciones_2

In [ ]:
# fs = [0.1, 0.5, 1]
# X = data[["x", "y"]]
# loess = Loess.Loess(X, data[["plano"]], 1)
# for f in fs:
#     print(f)
#     for superficie in superficies:
#         print(superficie)
#         agregar_estimacion(loess, data, superficie, f)
#data.to_csv("data_sintetica.csv")
data = pd.read_csv("data_sintetica.csv")

In [ ]:
fig = plt.figure(figsize=(30, 30))
titulos = ["z = -3x+y", "z = (x²)-(y²)", "z = -(x³)+y", "z = |x|+|y|"]
for i, superficie in enumerate(superficies):
    ax = fig.add_subplot(1, 4, i+1, projection='3d')
    zz = np.array(data[superficie]).reshape(xx.shape)
    ax.plot_surface(xx, yy, zz, cmap="plasma")
    ax.title.set_text(titulos[i])
    ax.title.set_size(30)

In [ ]:
fig = plt.figure(figsize=(30, 30))
for i, superficie in enumerate(superficies):
    ax = fig.add_subplot(1, 4, i+1, projection='3d')
    zz = np.array(data[f"{superficie}_ruido"]).reshape(xx.shape)
    ax.plot_surface(xx, yy, zz, cmap="plasma")

In [ ]:
fig = plt.figure(figsize=(30, 30))
for i, superficie in enumerate(superficies):
    ax = fig.add_subplot(1, 4, i+1, projection='3d')
    zz = np.array(data[f"estimacion_{superficie}_0.5_2"]).reshape(xx.shape)
    ax.plot_wireframe(xx, yy, zz, cmap="plasma")

In [ ]:
def curva_nivel(data, superficie, axis):
    fig, axs = plt.subplots(1, 2)
    fig.set_size_inches(13, 5)
    idx = 24

    zz = np.array(data[[f"estimacion_{superficie}_{0.5}_1"]]).reshape(xx.shape)
    if axis == "x":
        axs[1].plot(xx[0], zz[idx], color="red")
        axs[0].plot(xx[0], zz[idx], color="red")
    elif axis == "y":
        axs[1].plot(xx[0], zz.T[idx], color="red")
        axs[0].plot(xx[0], zz.T[idx], color="red")

    zz = np.array(data[[f"estimacion_{superficie}_{0.5}_2"]]).reshape(xx.shape)
    if axis == "x":
        axs[1].plot(xx[0], zz[idx], color="blue")
        axs[0].plot(xx[0], zz[idx], color="blue")
    elif axis == "y":
        axs[1].plot(xx[0], zz.T[idx], color="blue")
        axs[0].plot(xx[0], zz.T[idx], color="blue")

    zz = np.array(data[[superficie]]).reshape(xx.shape)
    if axis == "x":
        axs[0].plot(xx[0], zz[idx], color="black", linestyle="--")
    elif axis == "y":
        axs[0].plot(xx[0], zz.T[idx], color="black", linestyle="--")

    zz = np.array(data[[f"{superficie}_ruido"]]).reshape(xx.shape)
    if axis == "x":
        axs[1].scatter(xx[0], zz[idx], color='black')
    elif axis == "y":
        axs[1].scatter(xx[0], zz.T[idx], color='black')

    axs[0].legend(["lineal", "cuadratico", "curva real"])
    axs[1].legend(["lineal", "cuadratico", "data"])
    axs[0].set_xlabel(axis)
    axs[1].set_xlabel(axis)
    axs[0].set_ylabel("z")
    axs[1].set_ylabel("z")

In [ ]:
curva_nivel(data, "cubica", "x")

In [ ]:
curva_nivel(data, "cubica", "y")